corrected enhanced

In [2]:
from gtts import gTTS

text = "Once upon a time.."
tts = gTTS(text=text, lang="en")

tts.save("sample.mp3")


In [3]:
import whisper
import whisper.model
whisper.model.MultiHeadAttention.use_sdpa = False

model = whisper.load_model("tiny.en")

result = model.transcribe("sample.mp3")

# Print clean text (already cleaned by Whisper)
print(result["text"].strip())


c:\Users\muthuvu\Downloads\reauthor-whisper-asr-android-runtime-master\.venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
import numpy as np
import whisper
import onnxruntime as ort

# tokenizer
tokenizer = whisper.tokenizer.get_tokenizer(True)

# load audio → mel
audio = whisper.load_audio("sample.mp3")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).unsqueeze(0)
mel = mel.cpu().numpy().astype(np.float32)

# load ONNX
enc = ort.InferenceSession("encoder.onnx")
dec = ort.InferenceSession("decoder.onnx")

# encoder forward
audio_features = enc.run(None, {"mel": mel})[0].astype(np.float32)

# KV cache init
cache_self = np.zeros((8,1,1,384), np.float32)
cache_cross = np.zeros((8,1,1,384), np.float32)

# PREFILL cross-attn cache (CRITICAL)
dummy = np.array([[tokenizer.sot]], dtype=np.int64)
_, _, cache_cross = dec.run(
    None,
    {
        "tokens": dummy,
        "audio": audio_features,
        "cache_self_attn": cache_self,
        "cache_cross_attn": cache_cross,
    },
)

# reset self cache only
cache_self[:] = 0

# start decoding
tokens_list = [tokenizer.sot]
tokens = np.array([[tokenizer.sot]], dtype=np.int64)

logits, cache_self, cache_cross = dec.run(
    None,
    {
        "tokens": tokens,
        "audio": audio_features,
        "cache_self_attn": cache_self,
        "cache_cross_attn": cache_cross,
    },
)

# incremental decode
for _ in range(200):
    next_token = logits[:, -1, :].argmax(-1).astype(np.int64).reshape(1,1)

    if next_token[0,0] == tokenizer.eot:
        break

    tokens_list.append(int(next_token[0,0]))

    logits, cache_self, cache_cross = dec.run(
        None,
        {
            "tokens": next_token,
            "audio": audio_features,
            "cache_self_attn": cache_self,
            "cache_cross_attn": cache_cross,
        },
    )

# decode text with skip_special_tokens=True to remove special tokens
clean_text = tokenizer.decode(tokens_list, skip_special_tokens=True)
print(clean_text.strip())
